### Prueba de entrada

Un pipeline de NLP es una secuencia estructurada de pasos que permite a una computadora procesar y analizar texto de manera sistemática, extrayendo información significativa. Cada paso en el pipeline se enfoca en una tarea específica del procesamiento del lenguaje, y juntos forman un sistema completo para entender y manipular el lenguaje natural.

Los pasos fundamentales en un pipeline de NLP incluyen:

1. **Segmentación de oraciones:** Dividir el texto en oraciones individuales, lo cual facilita el procesamiento posterior al tratar cada oración como una unidad de análisis.
2. **Tokenización:** Separar las oraciones en palabras o tokens, lo que permite el análisis detallado de cada unidad de texto.
3. **Predicción de partes gramaticales (POS):** Identificar la categoría gramatical de cada token (como sustantivo, verbo, adjetivo), lo cual ayuda a comprender la estructura gramatical de las oraciones.
4. **Lematización:** Reducir las palabras a su forma básica o lema, para asegurar que diferentes formas de una palabra se traten como una misma entidad.
5. **Identificación de stopwords:** Marcar palabras comunes y de poca relevancia (como "and", "the"), que pueden ser filtradas para reducir el ruido en el análisis.
6. **Análisis de dependencias:** Entender las relaciones gramaticales entre las palabras de una oración, como identificar qué palabras dependen de otras.
7. **Reconocimiento de entidades nombradas (NER):** Detectar y etiquetar nombres de personas, lugares y organizaciones, para extraer información específica del texto.
8. **Resolución de coreferencia:** Determinar a qué entidad se refieren los pronombres en el texto, para entender mejor la referencia de entidades en el contexto.

Cada uno de estos pasos contribuye a la extracción de información significativa de un texto, permitiendo a una computadora "entender" el contenido de manera más efectiva. Este proceso es fundamental en muchas aplicaciones de NLP, como la traducción automática, la generación de texto, y el análisis de sentimientos.

**Nota:** Este proyecto deberá ser subido a tu repositorio personal a más tardar el 7 de septiembre.


**Pregunta**

Desarrolla un pipeline simplificado de NLP que incluya tres componentes clave: tokenización usando expresiones regulares (Regex), codificación Byte Pair Encoding (BPE) para la compresión de tokens, y cálculo de distancia de edición para comparar la similitud entre tokens.

**Instrucciones:**

1. **Tokenización con regex:**
   - Implementa un tokenizador utilizando expresiones regulares. El tokenizador debe ser capaz de separar el texto en palabras, respetando la puntuación como tokens independientes.
   - Por ejemplo, usando el texto: `"London is the capital and most populous city of England and the United Kingdom."`, el resultado esperado sería:
     ```
     ["London", "is", "the", "capital", "and", "most", "populous", "city", "of", "England", "and", "the", "United", "Kingdom", "."]
     ```

2. **Aplicación de byte pair encoding (BPE):**
   - Implementa el algoritmo de Byte Pair Encoding para comprimir la representación de los tokens.
   - Aplica BPE sobre el conjunto de tokens obtenido en el paso anterior y muestra cómo el vocabulario se reduce mediante la combinación de pares de caracteres más frecuentes.
   - Evalúa cómo BPE afecta el tamaño del vocabulario y la representación de palabras raras.

3. **Cálculo de distancia de edición:**
   - Implementa un algoritmo de distancia de edición (por ejemplo, la distancia de Levenshtein) para comparar la similitud entre diferentes tokens en el texto.
   - Utiliza la distancia de edición para identificar palabras similares dentro del texto tokenizado. Por ejemplo, compara palabras como `"London"` y `"Londinium"` y analiza su similitud.

4. **Integración en un pipeline:**
   - Integra los tres componentes anteriores en un pipeline simple que procese un texto de entrada desde la tokenización hasta la evaluación de similitudes.
   - Evalúa el desempeño del pipeline en un texto de prueba, y discute posibles mejoras o ajustes.

**Criterios de evaluación:**

- Correctitud en la implementación de cada componente del pipeline.
- Eficiencia en la aplicación del algoritmo BPE y en la reducción del vocabulario.
- Precisión en el cálculo de la distancia de edición y en la identificación de palabras similares.
- Calidad del código (documentación, modularidad, legibilidad) y análisis crítico de los resultados.


In [23]:
import regex as re

# Dividir oraciones en palabras
def sentence_segmentation(sentence):
    pattern = re.compile(r'[¿¡]?\b\w*[.!?]?\b')
    return [word for word in pattern.findall(sentence) if word.strip()]

In [17]:
# Intento de mejorar/corregir el algoritmo visto en clase

class BytePairEncoding:
    def __init__(self, num_merges):
        self.num_merges = num_merges
        self.vocab = {}
        self.merges = []

    def train(self, corpus):
        # Inicializamos el vocabulario con las palabras separadas por caracteres y añadimos el token de fin de palabra
        self.vocab = {}
        tokenized_corpus = [' '.join(list(word)) + ' </w>' for word in corpus]  # Añadimos el token de fin de palabra '</w>'

        # Contamos la frecuencia de cada palabra en el corpus tokenizado
        for word in tokenized_corpus:
            if word in self.vocab:
                self.vocab[word] += 1
            else:
                self.vocab[word] = 1

        print(f"Vocabulario inicial: {self.vocab}")

        # Realizamos los merges del BPE
        for merge_step in range(self.num_merges):
            pairs = self.get_pair_frequencies()

            if not pairs:
                break

            # Encontramos el par más frecuente
            most_frequent_pair = max(pairs, key=pairs.get)
            self.merges.append(most_frequent_pair)

            # Actualizamos el vocabulario uniendo los pares más frecuentes
            self.vocab = self.merge_vocabulary(most_frequent_pair)

            print(f"\nPaso {merge_step + 1}:")
            print(f"Par más frecuente: {most_frequent_pair}")
            print(f"Vocabulario actualizado: {self.vocab}")

    def get_pair_frequencies(self):
        """Obtiene las frecuencias de cada par de símbolos en el vocabulario."""
        pairs = {}
        for word, freq in self.vocab.items():
            tokens = word.split()
            for i in range(len(tokens) - 1):
                pair = (tokens[i], tokens[i + 1])
                if pair in pairs:
                    pairs[pair] += freq
                else:
                    pairs[pair] = freq
        return pairs

    def merge_vocabulary(self, pair):
        """Une los pares de símbolos más frecuentes en el vocabulario."""
        new_vocab = {}
        bigram = ' '.join(pair)  # Formamos el bigrama como una cadena

        # Reemplazamos los pares más frecuentes con el nuevo token
        for word in self.vocab:
            new_word = word.replace(bigram, ''.join(pair))  # Reemplazamos el par con el token unido
            new_vocab[new_word] = self.vocab[word]

        return new_vocab

    def tokenize(self, word):
        """Tokeniza una palabra basándose en las fusiones aprendidas por BPE."""
        tokens = list(word) + ['</w>']  # Añadimos el token de fin de palabra '</w>'
        print(f"\nPalabra original: {tokens}")

        # Convertimos la palabra en su representación BPE paso a paso
        while True:
            # Convertimos a cadena con espacios para facilitar la búsqueda de los merges
            current_word = ' '.join(tokens)
            applied = False

            # Intentamos aplicar cada regla de merge en el orden en que fue aprendida
            for merge in self.merges:
                bigram = ' '.join(merge)
                if bigram in current_word:
                    # Aplicamos la unión reemplazando el bigrama por el nuevo token
                    current_word = current_word.replace(bigram, ''.join(merge))
                    tokens = current_word.split()
                    applied = True
                    break

            # Si no se puede aplicar más merges, salimos del bucle
            if not applied:
                break

        # Removemos el token de fin de palabra '</w>' para la tokenización final
        if tokens[-1] == '</w>':
            tokens = tokens[:-1]

        return tokens

    def get_vocabulary(self):
        """Devuelve el vocabulario actual."""
        return self.vocab

# Ejemplo de uso
bpe = BytePairEncoding(num_merges=10)
corpus = ["low", "lowest", "lower", "lowering", "newest", "wider"]
bpe.train(corpus)

print("\nVocabulario final:", bpe.get_vocabulary())

# Ejemplos de tokenización
print("\nTokenización de 'lowest':", bpe.tokenize('lowest'))
print("\nTokenización de 'lowering':", bpe.tokenize('lowering'))
print("\nTokenización de 'newest':", bpe.tokenize('newest'))

Vocabulario inicial: {'l o w </w>': 1, 'l o w e s t </w>': 1, 'l o w e r </w>': 1, 'l o w e r i n g </w>': 1, 'n e w e s t </w>': 1, 'w i d e r </w>': 1}

Paso 1:
Par más frecuente: ('l', 'o')
Vocabulario actualizado: {'lo w </w>': 1, 'lo w e s t </w>': 1, 'lo w e r </w>': 1, 'lo w e r i n g </w>': 1, 'n e w e s t </w>': 1, 'w i d e r </w>': 1}

Paso 2:
Par más frecuente: ('lo', 'w')
Vocabulario actualizado: {'low </w>': 1, 'low e s t </w>': 1, 'low e r </w>': 1, 'low e r i n g </w>': 1, 'n e w e s t </w>': 1, 'w i d e r </w>': 1}

Paso 3:
Par más frecuente: ('low', 'e')
Vocabulario actualizado: {'low </w>': 1, 'lowe s t </w>': 1, 'lowe r </w>': 1, 'lowe r i n g </w>': 1, 'n e w e s t </w>': 1, 'w i d e r </w>': 1}

Paso 4:
Par más frecuente: ('s', 't')
Vocabulario actualizado: {'low </w>': 1, 'lowe st </w>': 1, 'lowe r </w>': 1, 'lowe r i n g </w>': 1, 'n e w e st </w>': 1, 'w i d e r </w>': 1}

Paso 5:
Par más frecuente: ('st', '</w>')
Vocabulario actualizado: {'low </w>': 1, 'lowe s

In [20]:
class LevenshteinDistance:
    @staticmethod
    def calculate(s1, s2):
        """Calcula la distancia de Levenshtein entre dos cadenas s1 y s2."""
        m = len(s1)
        n = len(s2)

        # Crear una matriz de (m+1) x (n+1) para almacenar distancias
        dp = [[0] * (n + 1) for _ in range(m + 1)]

        # Inicializar la primera fila y la primera columna
        for i in range(m + 1):
            dp[i][0] = i
        for j in range(n + 1):
            dp[0][j] = j

        # Llenar la matriz
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if s1[i - 1] == s2[j - 1]:
                    cost = 0
                else:
                    cost = 1
                dp[i][j] = min(dp[i - 1][j] + 1,      # Eliminación
                               dp[i][j - 1] + 1,      # Inserción
                               dp[i - 1][j - 1] + cost)  # Sustitución

        # El valor en dp[m][n] es la distancia de Levenshtein
        return dp[m][n]

# Ejemplo de uso:
levenshtein = LevenshteinDistance()
s1 = "London"
s2 = "Londinium"
distancia = levenshtein.calculate(s1, s2)

print(f"Distancia de Levenshtein entre '{s1}' y '{s2}' es {distancia}")

# https://es.wikipedia.org/wiki/Distancia_de_Levenshtein

Distancia de Levenshtein entre 'London' y 'Londinium' es 4


In [25]:
from itertools import combinations

# Pipeline principal
def process_text(text, bpe, levenshtein):
    print(f"Texto de entrada: {text}")

    # Paso 1: Segmentación de palabras
    words = sentence_segmentation(text)
    print(f"\nSegmentación de palabras: {words}")

    # Paso 2: Tokenización BPE
    tokenized_words = [bpe.tokenize(word) for word in words]
    print(f"\nTokenización BPE de cada palabra:")
    for word, tokens in zip(words, tokenized_words):
        print(f"'{word}': {tokens}")

    # Paso 3: Evaluación de similitud
    print("\nDistancias de Levenshtein entre todas las combinaciones de palabras:")
    for word1, word2 in combinations(words, 2):
        dist = levenshtein.calculate(word1, word2)
        print(f"Distancia entre '{word1}' y '{word2}': {dist}")

In [26]:
test_text = "London is the capital and most populous city of England and the United Kingdom."
bpe = BytePairEncoding(num_merges=10)
levenshtein = LevenshteinDistance()
process_text(test_text, bpe, levenshtein)

Texto de entrada: London is the capital and most populous city of England and the United Kingdom.

Segmentación de palabras: ['London', 'is', 'the', 'capital', 'and', 'most', 'populous', 'city', 'of', 'England', 'and', 'the', 'United', 'Kingdom']

Palabra original: ['L', 'o', 'n', 'd', 'o', 'n', '</w>']

Palabra original: ['i', 's', '</w>']

Palabra original: ['t', 'h', 'e', '</w>']

Palabra original: ['c', 'a', 'p', 'i', 't', 'a', 'l', '</w>']

Palabra original: ['a', 'n', 'd', '</w>']

Palabra original: ['m', 'o', 's', 't', '</w>']

Palabra original: ['p', 'o', 'p', 'u', 'l', 'o', 'u', 's', '</w>']

Palabra original: ['c', 'i', 't', 'y', '</w>']

Palabra original: ['o', 'f', '</w>']

Palabra original: ['E', 'n', 'g', 'l', 'a', 'n', 'd', '</w>']

Palabra original: ['a', 'n', 'd', '</w>']

Palabra original: ['t', 'h', 'e', '</w>']

Palabra original: ['U', 'n', 'i', 't', 'e', 'd', '</w>']

Palabra original: ['K', 'i', 'n', 'g', 'd', 'o', 'm', '</w>']

Tokenización BPE de cada palabra:
'

En cuanto al análisis, si bien se intentó mejorar el algoritmo visto en clase no pude hacerlo, es un pequeño problema de lógica así que la mejora empieza por ahí.
Por otro lado se ha establecido el pipeline de tal forma que acepta inyección de dependencias por si se quiere cambiar a otro tipo de algoritmo de tokenización como podría ser el caso del Scaffold-BPE, al igual que otro tipo de disntancia diferente a levenshtein.
Sin embargo, aunque las mejoras están en el pipeline, el punto más flojo de este código está en mi implementación del algoritmo BPE, con un poco más de trabajo daría mejores resultados.
Además, detalle importante es que si bien hay un pequeño tweak para que intente trabajar con oraciones en español al menos en la separación de oraciones, el pipeline solo ha sido pensado para el idioma inglés.